In [1]:
import sys
sys.path.append('../')
import wrangle

import nlp

from sklearn.decomposition import LatentDirichletAllocation, NMF

In [23]:
df, data_dict = wrangle.wrangle_data()

In [20]:
qual_questions = ['research_educ', 'research_educ_desc', 'how_pick_events', 'best_event',
       'events_attend_recent', 'ideal_conference_size', 'ideal_structure',
       'other_conference_types', 'ideal_topics', 'ideal_attendees',
       'recommendations']

In [32]:
df_qual = df.select_dtypes(include='object')

In [24]:
persona_id = df['persona_id']

In [33]:
df_qual.columns

Index(['research_educ', 'research_educ_desc', 'how_pick_events', 'best_event',
       'events_attend_recent', 'ideal_conference_size', 'ideal_structure',
       'other_conference_types', 'ideal_topics', 'ideal_attendees',
       'recommendations'],
      dtype='object')

In [34]:
df_qual = df_qual.fillna('n/a')

In [36]:
df_qual = df_qual.astype('str')

In [29]:
df_qual['persona_id'] = persona_id

In [37]:
df_qual['big_answer'] = df_qual['research_educ'] + df_qual['research_educ_desc'] + df_qual['how_pick_events'] + df_qual['best_event'] + df_qual['events_attend_recent'] + df_qual['ideal_conference_size'] + df_qual['ideal_structure'] + df_qual['other_conference_types'] + df_qual['ideal_topics'] + df_qual['ideal_attendees'] + df_qual['recommendations']

In [ ]:
df_q